# Q Agent

In [7]:
import random
from NN_models.Q_model import DQN as Q_model
from torch.optim import Adam
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
from torchvision.transforms import ToPILImage
import torch
import thread
import os
import json
import numpy as np
from PIL import Image
import glob
use_GPU = torch.cuda.is_available()
PILMODE='L'
Image_Size=[240,240]
input_shape = [24,240,240]
actionsize=6
loss_func=nn.MSELoss()
if use_GPU:
    loss_func = loss_func.cuda()

In [3]:

class DQNAgent:
    model_path='DQN_models'
    memory_path='RL_DATA/'
    state_file=memory_path+'STATE'
    action_file=memory_path+'ACTION'
    ep_reward_file=memory_path+'ep_reward.dat'
    batch_size = 25
    epsilon = 1
    epsilon_decay = 0.99
    epsilon_final = 0.1
    epsilon_endtime = 30000
    action_size = 6
    discount_factor = 0.7
    n_replay = 1 #replay per learning step
    clip_delta = 1

    def __init__(self,episode=1):
        self.ep = episode
        if episode==1:
            self.F_model = Q_model(output_shape=actionsize)
            self.target_F_model = Q_model(output_shape=actionsize)
        else:
            self.load_model(episode)
        if use_GPU:
            self.F_model.cuda()
            self.target_F_model.cuda()
        self.F_optimizer = Adam(self.F_model.parameters(),1e-4)
    
    def get_action(self,state):
        nsize = [1,]
        nsize.extend(state.shape)
        state = state.reshape(nsize)
        state = Variable(torch.Tensor(state))
        state = torch.Tensor(state)
        if use_GPU:
            state = state.cuda()
        res = self.Y_model(state)
        _,act = res.max()
        return max
    def get_action_eps(self,state):
        if random.random() <= self.epsilon:
            #if self.epsilon > self.epsilon_final:
                #self.epsilon*=self.epsilon_decay
            return random.randint(0,self.action_size-1)
        return self.get_action(state)
            
    
    def memorize(self,state,action,n_state,ep,ts):
        thread.start_new_thread(save_state,(state,ep,ts))
        thread.start_new_thread(save_action,(action,ep,ts))

    def save_model(self,tag=None):
        name='episode_%d_model'%self.ep
        if tag is not None:
            name += tag
        torch.save(self.target_F_model,name+'.model')
        
    def load_model(self):
        name='episode_%d_model'%(self.ep-1)
        self.F_model = torch.load(name)
        self.target_F_model=torch.load(name)
    
    def update_target_model(self):
        self.target_F_model.load_state_dict(self.F_model.state_dict())
        
    def experience_replay(self,N=10,batchsize=25):
        memory = load_memory_of_episode(self.ep)
        for i in range(int(N*len(memory)/batchsize)):
            minibatch = random.sample(memory,batchsize)
            shape=[batchsize,]
            shape.extend(input_shape)
            update_input = np.zeros(shape)
            update_target = np.zeros((batchsize, self.action_size))
            for j in range(batchsize):
                state = minibatch[j]['state']
                action = int(minibatch[j]['action'])
                reward = minibatch[j]['reward']
                n_state = minibatch[j]['n_state']
                
                ystate = Variable(torch.FloatTensor(state))
                nstate = Variable(torch.FloatTensor(n_state))
                nshape=[1,]
                nshape.extend(ystate.size())
                ystate = ystate.view(nshape)
                nstate = nstate.view(nshape)
                if use_GPU:
                    ystate = ystate.cuda()
                    nstate = nstate.cuda()
                target = self.F_model.forward(ystate)[0].cpu().data.numpy()
                q_2 =self.target_F_model.forward(nstate)
                q_2_max = torch.max(q_2).cpu().data.numpy()
                target[action] = reward + self.discount_factor*q_2_max

                update_input[i] = state
                update_target[i] = target
            
            update_input=Variable(torch.FloatTensor(update_input))
            update_target=Variable(torch.FloatTensor(update_target))
            if use_GPU:
                update_input=update_input.cuda()
                update_target=update_target.cuda()
            prediction=self.F_model.forward(update_input)
            loss = loss_func(prediction,update_target)

            self.F_model.zero_grad()
            loss.backward()
            
            for param in self.F_model.parameters():
                param.grad.data.clamp_(-1,1)
            self.F_optimizer.step()
        self.update_target_model()
    
def save_state(state,ep,time_stamp):
    s_dir='RL_DATA/EP%d/STATE/s%04d'%(ep,time_stamp)
    if not os.path.isdir(s_dir):
        os.makedirs(s_dir)
    for i in range(state.shape[0]):
        frame=ToPILImage(state[i])
        frame.save(s_dir+'/%04d.png'%i)
def save_action(action,ep,time_stamp):
    s_dir='RL_DATA/EP%d/ACTION'%ep
    if not os.path.isdir(s_dir):
        os.makedirs(s_dir)
    actfil=open(s_dir+'/a%04d.data'%time_stamp,'w')
    actfil.write(str(action))
    actfil.close()

In [4]:
state = np.zeros((24,480,640))
nsize = [1,]
nsize.extend(state.shape)
state = state.reshape(nsize)
state = Variable(torch.Tensor(state))

In [5]:
a = Q_model()

In [8]:
torch.argmax(a(state))

tensor(2)

In [13]:
# train
import random
from torch.optim import Adam
from torch.autograd import Variable
import torch
import glob
from model import *
from PIL import Image
import numpy as np
import json
use_GPU=torch.cuda.is_available()
D=torch.load('iter_4650.model')
C=torch.load('CNN_8613.model')#load classifier
if use_GPU:
    C.cuda()
    D.cuda()
D.eval()
C.eval()
N=100
episode=int(open('RL_DATA/EP.data').readline())
w_c=0.5
w_d=1
d_thres=0.2
Image_Size=240
UImage_Size=784
input_shape = [24,240,240]
def reform_one(data):
    xl=data[:,::3]
    yl=data[:,1::3]
    add_dim = lambda x:x.reshape(x.shape[0],1,-1,1)
    xl=add_dim(xl)
    yl=add_dim(yl)
    l = np.concatenate([xl,yl],3)
    score=data[:,2::3]
    score=score.reshape(score.shape[0],2,-1)
    return l,score
def normalization(cood,size):
    cood=cood/size-0.5
    return cood

In [42]:
def load_memory_of_episode(episode):
    directory = 'RL_DATA/EP%d'%episode
    #load states
    s_dir = directory +'/STATE/'
    s_list = glob.glob(s_dir+'s*')
    s_list.sort()
    states=list()
    for folder in s_list:
        frames = glob.glob(folder+'/*.png')
        frames.sort()
        state_list=list()
        for frame in frames:
            img = Image.open(open(frame))
            img_arr = np.array(img)
            state_list.append(img_arr)
        states.append(np.array(state_list))
    states = np.array(states)
    #load actions
    a_dir = directory +'/ACTION/'
    a_list = glob.glob(a_dir+'*')
    a_list.sort()
    actions = list()
    for fil in a_list:
        afile = open(fil)
        action = int(afile.readline())
        afile.close()
        actions.append(action)
    #calculate reward
    pose_list = s_list
    pose_list.sort()
    poses_o=list()
    person_max=list()
    for folder in pose_list:
        posefils = glob.glob(folder+'/*.json')
        posefils.sort()
        pose_list=list()
        p_max=0#count person
        for posefil in posefils:
            frame=json.load(open(posefil))
            if len(frame['people']) >0:
                p_max = len(frame['people'])
                pose=frame['people'][0]['pose_keypoints_2d']
            else:
                pose = [Image_Size/2 for i in range(54)]
            pose_list.append(pose)
        pose_list = np.array(pose_list)
        original,score=reform_one(pose_list)
        processed=normalization(original,Image_Size)
        poses_o.append(processed)
    poses_o=np.array(poses_o)
    #get action skeleton
    action_skeleton_folder = 'robot/action_skeletons/'
    poses_s=list()
    for act in actions:
        fildir = action_skeleton_folder+str(act)
        posefils = glob.glob(fildir+'/*.json')
        posefils.sort()
        pose_list=list()
        for posefil in posefils[:24]: # only need 24 frames
            frame=json.load(open(posefil))
            if len(frame['people']) >0:
                pose=frame['people'][0]['pose_keypoints_2d']
            else:
                pose = [UImage_Size/2 for i in range(54)]
            pose_list.append(pose)
        pose_list = np.array(pose_list)
        original,score=reform_one(pose_list)
        processed=normalization(original,UImage_Size)
        poses_s.append(processed)
    poses_s=np.array(poses_s)
    pair = np.concatenate((poses_o[1:],poses_s[:-1]),axis=2)
    pair = Variable(torch.Tensor(pair))
    if use_GPU:
        pair = pair.cuda()
#######################################################################todo
    C_score = F.softmax(C(pair)).cpu().data.numpy()
    m=np.max(C_score,axis=1)
    agm=np.argmax(C_score,axis=1)
    for i in range(len(agm)):
        if agm[i]==7:# recognized as unknown interaction
            m[i]=0
    C_reward=m
    D_reward = F.sigmoid((D(pair)[0] -0.2)).cpu().data.numpy().reshape(-1)
    reward =  w_c*C_reward+w_d*D_reward
    print(reward)
#######################################################################
    memories = list()
    for i in range(len(actions)-1):
        memories.append(dict(state =states[i],action=actions[i],reward=reward[i],n_state=states[i+1]))
    return memories

In [22]:
model = DQNAgent(episode=episode)

NameError: name 'DQNAgent' is not defined

In [52]:
model.experience_replay()

In [212]:
a = np.ones((100,10),dtype=int)

In [43]:
mem = load_memory_of_episode(1)

[1.2268616 1.3444424 1.4092128 1.3178624]


/home/jiao/miniconda3/envs/pepper/lib/python2.7/site-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [41]:
mem[1]['reward']

2.1916597